In [1]:
# Demo recsys boolean (0-1)

# import libraries

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from operator import itemgetter
import plotly.plotly as py
import cufflinks as cf
cf.go_offline()

In [2]:
# names and tags

names=['id__'+str(i) for i in range(3000)]

tags=['r&b', 'rock', 'jazz', 'electronic', 'pop', 'indie',
      'cinema', 'theater', 'beers', 'wine', 'party', 'trips',
      'running', 'gym', 'golf', 'basket', 'football', 'yoga']


sub_tags_elec=['techno', 'electro_funky', 'house', 'minimal', 'dubstep', 'DandB']

In [3]:
# synth data

data=np.random.randint(0, 2, (len(names), len(tags)))*0.8
sub_data=np.random.randint(0, 1, (len(names), len(sub_tags_elec)))

s_data=pd.DataFrame(data, columns=tags, index=names)
s_sub_data=pd.DataFrame(sub_data, columns=sub_tags_elec, index=names)

s_data=pd.concat([s_data, s_sub_data], axis=1)

s_data['plan']=np.random.randint(0, 2, (len(names), 1))
s_data['id']=[i for i in range(len(names))]


for i in range(len(s_data)):
    if s_data.electronic[i]!=0:
        s_data.techno[i]=np.random.randint(0, 2)
        s_data.electro_funky[i]=np.random.randint(0, 2)
        s_data.house[i]=np.random.randint(0, 2)
        s_data.minimal[i]=np.random.randint(0, 2)
        s_data.dubstep[i]=np.random.randint(0, 2)
        s_data.DandB[i]=np.random.randint(0, 2)

In [52]:
#  weighing

for e in tags:
    s_data[e]=s_data[e]*2/3

for e in sub_tags_elec:
    s_data[e]=s_data[e]*1/3
    
display(s_data.tail())

,r&b,rock,jazz,electronic,pop,indie,cinema,theater,beers,wine,...,football,yoga,techno,electro_funky,house,minimal,dubstep,DandB,plan,id
id__2995,0.000000,0.000000,0.158025,0.158025,0.158025,0.000000,0.158025,0.000000,0.158025,0.000000,...,0.158025,0.000000,0.000000,0.0,0.0,0.000000,0.012346,0.000000,1,2995
id__2996,0.158025,0.000000,0.000000,0.000000,0.158025,0.000000,0.000000,0.158025,0.000000,0.000000,...,0.000000,0.158025,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1,2996
id__2997,0.158025,0.158025,0.158025,0.158025,0.158025,0.000000,0.000000,0.158025,0.000000,0.158025,...,0.000000,0.000000,0.000000,0.0,0.0,0.012346,0.000000,0.012346,0,2997
id__2998,0.000000,0.158025,0.000000,0.158025,0.158025,0.158025,0.158025,0.158025,0.158025,0.000000,...,0.000000,0.000000,0.012346,0.0,0.0,0.000000,0.012346,0.012346,1,2998
id__2999,0.000000,0.000000,0.158025,0.000000,0.158025,0.158025,0.158025,0.158025,0.158025,0.000000,...,0.158025,0.158025,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1,2999


In [65]:
# new user function


def new_user(df, rb, rock, jazz, electronic, pop, indie, cinema, theater, beers, wine,
             party, trips, running, gym, golf, basket, football, yoga,
             techno, electro_funky, house, minimal, dubstep, DandB, metric):
    
    
    tags=['r&b', 'rock', 'jazz', 'electronic', 'pop', 'indie',
          'cinema', 'theater', 'beers', 'wine', 'party', 'trips',
          'running', 'gym', 'golf', 'basket', 'football', 'yoga',
          'techno', 'electro_funky', 'house', 'minimal', 'dubstep', 'DandB']
    
    
    
    rating=[rb, rock, jazz, electronic, pop, indie, cinema, theater, beers, wine,
            party, trips, running, gym, golf, basket, football, yoga, 
            techno, electro_funky, house, minimal, dubstep, DandB]
    
    
    n_user={k:v for k, v in list(zip(tags, rating))}
    n_user['id']=len(s_data.id)
    n_user['plan']=0
    
    names=list(df.index)
    df=df.append(n_user, ignore_index=True)
    names.append('id__'+str(n_user['id']))
    df.index=names
    
    similar = pd.DataFrame(1/(1 + squareform(pdist(df.iloc[:, :-2], metric))), 
                         index=df.index, columns=df.index)

    similarities = similar[n_user['id']].sort_values(ascending=False)
    
    closer_users=[]
    for e in similarities.index:
        if df.ix[e].plan==1:
            closer_users.append(e)
    
    return list(map(list, zip(closer_users[:10], similarities.ix[closer_users[:10]])))#df.ix[closer_users[:10]][['id', 'plan']]

In [6]:
print('Rate r&b :')
rb = int(input())*1.6/3

Rate r&b :
0


In [7]:
print('Rate rock :')
rock = int(input())*1.6/3  

Rate rock :
1


In [8]:
print('Rate jazz :')
jazz = int(input())*1.6/3 

Rate jazz :
1


In [9]:
print('Rate electronic :')
electronic = int(input())*1.6/3  

Rate electronic :
0


In [10]:
if electronic!=0:
    print('Rate techno :')
    techno = int(input())*0.8/3
    
    print('Rate electro-funky :')
    electro_funky = int(input())*0.8/3
 
    print('Rate house :')
    house = int(input())*0.8/3
 
    print('Rate minimal :')
    minimal = int(input())*0.8/3
 
    print('Rate dubstep :')
    dubstep = int(input())*0.8/3
 
    print('Rate D&B :')
    DandB = int(input())*0.8/3


else:
    techno = 0
    electro_funky = 0
    house = 0
    minimal = 0
    dubstep = 0
    DandB = 0

In [11]:
print('Rate pop from :')
pop = int(input())*1.6/3 

Rate pop from :
1


In [12]:
print('Rate indie :')
indie = int(input())*1.6/3 

Rate indie :
1


In [13]:
print('Rate cinema :')
cinema = int(input())*1.6/3 

Rate cinema :
1


In [14]:
print('Rate theater :')
theater = int(input())*1.6/3 

Rate theater :
0


In [15]:
print('Rate beers :')
beers = int(input())*1.6/3 

Rate beers :
0


In [16]:
print('Rate wine :')
wine = int(input())*1.6/3 

Rate wine :
1


In [17]:
print('Rate party :')
party = int(input())*1.6/3 

Rate party :
0


In [18]:
print('Rate trips :')
trips = int(input())*1.6/3 

Rate trips :
1


In [19]:
print('Rate running :')
running = int(input())*1.6/3 

Rate running :
1


In [20]:
print('Rate gym :')
gym = int(input())*1.6/3 

Rate gym :
0


In [21]:
print('Rate golf :')
golf = int(input())*1.6/3 

Rate golf :
0


In [22]:
print('Rate basket :')
basket = int(input())*1.6/3 

Rate basket :
1


In [23]:
print('Rate football :')
football = int(input())*1.6/3 

Rate football :
0


In [24]:
print('Rate yoga :')
yoga = int(input())*1.6/3 

Rate yoga :
1


In [25]:
# metrics for squareform

'''
'euclidean', 'minkowski', 'cityblock', 'seuclidean', 'sqeuclidean', 'cosine', 'correlation'

'hamming', 'jaccard', 'chebyshev', 'canberra', 'braycurtis', 'mahalanobis', 'yule'

'matching', 'dice', 'kulsinski', 'rogerstanimoto', 'russellrao', 'sokalmichener'

'sokalsneath', 'wminkowski'
'''


metric='cosine' 

In [66]:
# plans (users)

plans=new_user(s_data, rb, rock, jazz, electronic, pop, indie, cinema, theater, beers, wine,
               party, trips, running, gym, golf, basket, football, yoga, 
               techno, electro_funky, house, minimal, dubstep, DandB, metric)

#display (s_data)

print ('Similar users with plan : ')
for e in plans:
    print (e[0])
    
final=pd.DataFrame()
final['plan_ori_id']=[e[0] for e in plans]
final['original_similarity']=[e[1] for e in plans]

               r&b      rock      jazz  electronic       pop     indie  \
id__0     0.158025  0.158025  0.158025    0.158025  0.000000  0.000000   
id__1     0.158025  0.158025  0.000000    0.158025  0.158025  0.158025   
id__2     0.158025  0.000000  0.158025    0.000000  0.158025  0.000000   
id__3     0.158025  0.158025  0.158025    0.158025  0.158025  0.158025   
id__4     0.158025  0.000000  0.000000    0.000000  0.158025  0.158025   
id__5     0.158025  0.000000  0.000000    0.000000  0.158025  0.000000   
id__6     0.000000  0.158025  0.158025    0.158025  0.000000  0.000000   
id__7     0.000000  0.000000  0.000000    0.000000  0.158025  0.000000   
id__8     0.000000  0.158025  0.158025    0.000000  0.158025  0.158025   
id__9     0.000000  0.158025  0.000000    0.000000  0.158025  0.158025   
id__10    0.158025  0.158025  0.000000    0.000000  0.158025  0.000000   
id__11    0.000000  0.158025  0.000000    0.158025  0.158025  0.000000   
id__12    0.158025  0.158025  0.000000

KeyError: 3000

In [27]:
# follow-unfollow users
foll=plans
def follow(_id, b, foll):
    if b==0: return foll
    elif b==1:
        for e in foll:
            if _id==e[0]:
                e[1]+=e[1]*0.15
        for e in foll:
            if e[1]>1:e[1]=1
        return foll#sorted(foll, key=itemgetter(1), reverse=True)

print ()
for e in foll:
    print ('Follow user {}?'.format(e[0]))
    b=int(input())
    f=follow(e[0], b, foll)
    #print (f)
    print()
#final['follow_id']=[e[0] for e in f]
final['after_follow_similarity']=[e[1] for e in f]



unfoll=foll
def unfollow(_id, b, unfoll):
    if b==0: return unfoll
    elif b==1:
        for e in unfoll:
            if _id==e[0]:
                e[1]-=e[1]*0.15
        for e in unfoll:
            if e[1]<0:e[1]=0
        return unfoll#sorted(unfoll, key=itemgetter(1), reverse=True)

print ()
for e in unfoll:
    print ('Unfollow user {}?'.format(e[0]))
    b=int(input())
    u=unfollow(e[0], b, unfoll)
    #print (u)
    print()
#final['unfollow_id']=[e[0] for e in u]
final['after_unfollow_similarity']=[e[1] for e in u]
display (final)


Follow user id__1000?
1

Follow user id__559?
1

Follow user id__2058?
1

Follow user id__1514?
0

Follow user id__785?
0

Follow user id__991?
0

Follow user id__2159?
0

Follow user id__877?
0

Follow user id__2483?
0

Follow user id__2790?
0


Unfollow user id__1000?
0

Unfollow user id__559?
0

Unfollow user id__2058?
1

Unfollow user id__1514?
0

Unfollow user id__785?
0

Unfollow user id__991?
0

Unfollow user id__2159?
1

Unfollow user id__877?
0

Unfollow user id__2483?
1

Unfollow user id__2790?
0



,plan_ori_id,original_similarity,after_follow_similarity,after_unfollow_similarity
0,id__1000,0.727273,0.836364,0.836364
1,id__559,0.723607,0.832148,0.832148
2,id__2058,0.700597,0.805686,0.684833
3,id__1514,0.692733,0.692733,0.692733
4,id__785,0.691607,0.691607,0.691607
5,id__991,0.691607,0.691607,0.691607
6,id__2159,0.683280,0.683280,0.580788
7,id__877,0.678877,0.678877,0.678877
8,id__2483,0.678877,0.678877,0.577046
9,id__2790,0.678877,0.678877,0.678877


In [28]:
#def like():

In [29]:
final.iplot(kind='bar', x='plan_ori_id', title='Similarities with user id_3000', 
            yTitle='Similarity', xTitle='Users')